# Acquiring and Storing Data

## Assignment 5

For this assignment and future assignments, assume that you are the owner of a small but growing retail business, *Datums R Us*. Your store sells technology, tools, and clothing for the discerning data scientist. You currently have stores in the following five locations. 

- Bellevue, Nebraska
- Columbus, Ohio
- Denver, Colorado
- San Francisco, California
- Baltimore, Maryland

You have been tasked with creating a data lake for the company using a [directory structure based on Cookiecutter Data Science recommendations](https://drivendata.github.io/cookiecutter-data-science/#directory-structure). This basic directory structure works well for small, self-contained data science projects and organizing large-scale data warehouses.

```
├── data
│   ├── external       <- Data from third-party sources.
│   ├── interim        <- Intermediate data that has been transformed.
│   ├── processed      <- The final, canonical data sets for modeling and reports.
│   └── raw            <- The original, immutable data dump.
```

You have identified the following items for initial inclusion in the data lake. 

**External Data Sets**

- Census (Updated Yearly)
- Weather Forecasts (Updated Daily)

**Raw Data Dumps**

- Sales (Updated Hourly)
- Inventory (Updated Daily)
- Expenses (Updated Daily)

**Processed Data Sets and Reports**

*Weekly*

- Modeling Data Set

*Monthly*

- Inventory Update Request

*Quarterly*

- Quarterly Financial Report

### Assignment 5.1

In the first part of the assignment, you will describe the directory structure for the data lake. For the most part, this directory structure will not depend on the technical details of how you store the data. You could be storing the data in a local filesystem, a distributed filesystem such as HDFS, or object storage, such as Amazon S3. 

You will only be creating the directory structures and not populating actual content. Real-world data lakes store data in a variety of formats including,  Apache Parquet, Google Protocol Buffers, Apache Avro, JSONL, and CSV. 

You will use Python's built-in [calendar library](https://docs.python.org/3/library/calendar.html), and [datetime library](https://docs.python.org/3/library/datetime.html) to work with the dates and times required for this assignment. You will use the [PurePosixPath](https://docs.python.org/3/library/pathlib.html#pathlib.PurePosixPath) class from Python's built-in [pathlib library](https://docs.python.org/3/library/pathlib.html) to represent locations on the data lake. 

You will generate the output directories for an entire year's worth of data starting on January 1st of this year. Unless otherwise specified, all times will be in Coordinated Universal Time (UTC). 

In [9]:
# Imports the required Python libraries and 
# sets global variables for the assignment
import calendar
import datetime
from pathlib import PurePosixPath, Path
from functools import reduce

today = datetime.date.today()
current_year = today.year
days_in_year = 365

if calendar.isleap(current_year):
    days_in_year +=1

hours_in_year = days_in_year * 24

In [3]:
# Creates paths for the external, interim, processed, and raw directories
# Use these paths when creating new paths

root_data_dir = Path('data')
external_data_dir = root_data_dir.joinpath('external')
interim_data_dir = root_data_dir.joinpath('interim')
processed_data_dir = root_data_dir.joinpath('processed')
raw_data_dir = root_data_dir.joinpath('raw')

print('Root Data Directory: {}'.format(root_data_dir))
print('External Data Directory: {}'.format(external_data_dir))
print('Interim Data Directory: {}'.format(interim_data_dir))
print('Processed Data Directory: {}'.format(processed_data_dir))
print('Raw Data Directory: {}'.format(raw_data_dir))

Root Data Directory: data
External Data Directory: data\external
Interim Data Directory: data\interim
Processed Data Directory: data\processed
Raw Data Directory: data\raw


#### Assignment 5.1.a

For the purposes of this assignment, we will be using three Census data sets as examples of external data updated yearly. These data sets are:

- [American Community Survey (ACS) Summary File](https://www.census.gov/programs-surveys/acs/data/summary-file.html)
- [American Community Survey (ACS) Public Use Microdata Sample (PUMS)]( https://www.census.gov/programs-surveys/acs/microdata.html)
- [Tiger/Line Shapefiles](https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html)

If you are curious, you can find the actual data sets at the following locations: 

- [ACS Summary File](https://www2.census.gov/programs-surveys/acs/summary_file/)
- [PUMS](https://www2.census.gov/programs-surveys/acs/data/pums/)
- [Tiger](https://www2.census.gov/geo/tiger/)

For this assignment, we use the following naming convention for external data sets

```
/data/external/<source>/<data-set>/<year>/
```
where *source* is the organization providing the data, *data-set* is the specific data set, and *year* is the year. 

```
data
├── external
│   ├── census
│   │   ├── acs-summaryfile
│   │   │   ├── 2015
│   │   │   ├── 2016
│   │   │   ...
│   │   │   ...
│   │   │   └── 2019
│   │   ├── pums
│   │   │   ├── 2015
│   │   │   ├── 2016
│   │   │   ...
│   │   │   ...
│   │   │   └── 2020
│   │   └── tiger
│   │       ├── 2015
│   │       ├── 2016
│   │   │   ...
│   │   │   ...
│   │       └── 2020
│   └── nwc-wpc
├── interim
├── processed
└── raw
```

Create and add the paths for these data sets. Verify that you have added the paths correctly. 

In [4]:
acs_summary_file_dirs = set()

# TODO: Create and add the paths for this data set
census_path = external_data_dir.joinpath('census')
acs_path = census_path.joinpath('acs-summaryfile')

census_path.mkdir(parents=True, exist_ok=True)
acs_path.mkdir(exist_ok=True)


for yr in range(2015,current_year):
    yr_temp_path = acs_path.joinpath(str(yr))
    try:
        yr_temp_path.mkdir()
    except FileExistsError:
        pass
    except Exception as e:
        print(e)
for yr_path in acs_path.iterdir():
    acs_summary_file_dirs.add(yr_path)
# Should output sorted directories from 2015 to present 
sorted(list(acs_summary_file_dirs)) 

[WindowsPath('data/external/census/acs-summaryfile/2015'),
 WindowsPath('data/external/census/acs-summaryfile/2016'),
 WindowsPath('data/external/census/acs-summaryfile/2017'),
 WindowsPath('data/external/census/acs-summaryfile/2018'),
 WindowsPath('data/external/census/acs-summaryfile/2019'),
 WindowsPath('data/external/census/acs-summaryfile/2020'),
 WindowsPath('data/external/census/acs-summaryfile/2021'),
 WindowsPath('data/external/census/acs-summaryfile/2022')]

#### Assignment 5.1.b

In [5]:
pums_dirs = set()

# TODO: Create and add the paths for this data set
pums_path = census_path.joinpath('pums')

pums_path.mkdir(exist_ok=True)


for yr in range(2015,current_year):
    yr_temp_path = pums_path.joinpath(str(yr))
    try:
        yr_temp_path.mkdir()
    except FileExistsError:
        pass
    except Exception as e:
        print(e)
for yr_path in pums_path.iterdir():
    pums_dirs.add(yr_path)


# Should output sorted directories from 2015 to present 
sorted(list(pums_dirs)) 

[WindowsPath('data/external/census/pums/2015'),
 WindowsPath('data/external/census/pums/2016'),
 WindowsPath('data/external/census/pums/2017'),
 WindowsPath('data/external/census/pums/2018'),
 WindowsPath('data/external/census/pums/2019'),
 WindowsPath('data/external/census/pums/2020'),
 WindowsPath('data/external/census/pums/2021'),
 WindowsPath('data/external/census/pums/2022')]

#### Assignment 5.1.c

In [6]:
tiger_dirs = set()

# TODO: Create and add the paths for this data set
tiger_path = census_path.joinpath('tiger')

tiger_path.mkdir(exist_ok=True)

for yr in range(2015,current_year):
    yr_temp_path = tiger_path.joinpath(str(yr))
    try:
        yr_temp_path.mkdir()
    except FileExistsError:
        pass
    except Exception as e:
        print(e)
for yr_path in tiger_path.iterdir():
    tiger_dirs.add(yr_path)


# Should output sorted directories from 2015 to present 
sorted(list(tiger_dirs)) # Should output sorted directories from 2015 to present 

[WindowsPath('data/external/census/tiger/2015'),
 WindowsPath('data/external/census/tiger/2016'),
 WindowsPath('data/external/census/tiger/2017'),
 WindowsPath('data/external/census/tiger/2018'),
 WindowsPath('data/external/census/tiger/2019'),
 WindowsPath('data/external/census/tiger/2020'),
 WindowsPath('data/external/census/tiger/2021'),
 WindowsPath('data/external/census/tiger/2022')]

#### Assignment 5.1.d

Finally, you will create directories for a daily data set based on the [National Weather Service's (NWS) Weather Prediction Center's (WPC) daily forecasts](https://www.wpc.ncep.noaa.gov/kml/kmlproducts.php). 

For this part, we use the following naming convention

```
/data/external/nwc-wpc/forecasts/<year>/<month>/<day>/
```
where *year* is the year, *month* is the two-digit month, and *day* is the two-digit day. We use this convention when working with date-based data as the directories are naturally in date order. 

```
data
├── external
│   ├── census
│   └── nwc-wpc
│       └── forecasts
│           └── 2020
│               ├── 01
│               │   ├── 01
│               │   ├── 02
│               │   ├── 03
│               │   ...
│               │   ...
│               │   ├── 30
│               │   └── 31
│               ├── 02
│               │   ├── 01
│               │   ├── 02
│               │   ...
│               │   ...
│               │   ├── 28
│               │   └── 29
│               ├── 03
│               ...
│               ...
│               ├── 11
│               └── 12
│                   ├── 01
│                   ├── 02
│                   ...
│                   ...
│                   ├── 29
│                   ├── 30
│                   └── 31
├── interim
├── processed
└── raw
```

Create and add the paths for these data sets. Verify that you have added the paths correctly. 

In [30]:
nwc_wpc_path = external_data_dir.joinpath('nwc-wpc')
forecast_path = nwc_wpc_path.joinpath('forecasts')


# TODO: Create and add the paths for this data set
def get_day_cal(target_year): 
    'take in target year, return list of days'
    # First instantize a calender object and get a multi-dimensional array containing the months, weeks, and days for the given year.
    # We use reduce and lambda to flatten the array to a single dimension - retaining order. 
    # Since the calendar can contain some entries from the year prior, or the year following, due to the weeks - so we filter them out.
    # Finally, return the final list.
    year_data = calendar.Calendar().yeardatescalendar(target_year, width=12)
    month_list = reduce(lambda x,y: x+y, year_data)
    week_list = reduce(lambda x,y: x+y, month_list)
    day_list = reduce(lambda x,y: x+y, week_list)
    day_list = [target_day for target_day in day_list if target_day.year == target_year]
    return day_list

def gen_date_paths(date_list, base_path, day_of_week = False):
    '''Iterate through the date_list, then grab the parts of the datetime type and create a path object.
    Next we add the path to a set containing directories, then return the set.
    If day_of_week is True then it changes the day to the day of week instead'''
    cal_date_dirs = set()
    for date in date_list:
        year, month, day = date.year, date.month, date.day
        if day_of_week:
            day = calendar.weekday(year, month, day)
        date_tmp_path = base_path.joinpath(f'{year}/{month}/{day}')
        cal_date_dirs.add(date_tmp_path)
    return cal_date_dirs

def make_date_paths(path_list):
    'Attempts to make the directors for all paths in the list. Returns True if completed successfully, False if unsuccessful.'
    for path_ in path_list:
        try:
            path_.mkdir(parents=True)
        except FileExistsError:
            continue
        except Exception as e:
            print(e)
            return False
    return True
curr_days = get_day_cal(current_year)
forecast_dirs = gen_date_paths(curr_days, forecast_path)
make_date_paths(forecast_dirs)

next_yr_days = get_day_cal(current_year+1)
leap_dirs = gen_date_paths(next_yr_days, forecast_path)
make_date_paths(leap_dirs)
# Should have 365 directories (366 if leap year)

print(len(forecast_dirs), len(leap_dirs))

365 366


### Assignment 5.2

In the second part of the assignment, you will create the structure for the raw source data. We will use the following directory naming convention. 

```
/data/raw/inventory/<location>/<year>/<month>/<day>/
/data/raw/expenses/<location>/<year>/<month>/<day>/
/data/raw/sales/<location>/<year>/<month>/<day>/<hour>/
```
For *location*, we will use the three-letter IATA code for the airport nearest to the location.  We will use the same year, month, and day convention from the previous example. For *hour*, we will use the two-digit hour value based on a 24-hour clock set to UTC. 

#### Assignment 5.2.a

The following is an example of the directory structure for daily data dumps. 

```
data
├── external
├── interim
├── processed
└── raw
    ├── expenses
    ├── inventory
    │   ├── bwi
    │   ├── cmh
    │   ├── den
    │   ├── oma
    │   │   └── 2020
    │   │       ├── 01
    │   │       │   ├── 01
    │   │       │   ├── 02
    │   │       │   ...    
    │   │       │   └── 31
    │   │       ├── 02
    │   │       │   ├── 01
    │   │       │   ...
    │   │       │   └── 29
    │   │       ├── 03
    │   │       ... 
    │   │       ├── 11
    │   │       └── 12
    │   │           ├── 01
    │   │           ├── 02
    │   │           ...  
    │   │           └── 31
    │   └── sfo
    └── sales
```

Create and add the paths for these data sets. Verify that you have added the paths correctly.

In [23]:
inventory_dirs = set()
inventory_path = raw_data_dir.joinpath('inventory')
locations = ['bwi', 'cmh', 'den', 'oma', 'sfo']
inv_paths = [inventory_path.joinpath(loc) for loc in locations]
# TODO: Create and add the paths for this data set
for loc_path in inv_paths:
    temp_loc_set = gen_date_paths(curr_days, loc_path)
    inventory_dirs |= temp_loc_set

make_date_paths(inventory_dirs)

# Should have 1825 directories (1830 if leap year)
len(inventory_dirs) 

1825

In [24]:
expenses_dirs = set()
expense_path = raw_data_dir.joinpath('expenses')
exp_paths = [expense_path.joinpath(loc) for loc in locations]
# TODO: Create and add the paths for this data set

for loc_path in exp_paths:
    temp_loc_set = gen_date_paths(curr_days, loc_path)
    expenses_dirs |= temp_loc_set

make_date_paths(expenses_dirs)

# Should have 1825 directories (1830 if leap year)
len(expenses_dirs) 

1825

#### Assignment 5.2.b

Finally, create the paths for the hourly sales data. The following is an example of the directory structure for the sales data. 

```
├── external
├── interim
├── processed
└── raw
    ├── expenses
    ├── inventory
    └── sales
        ├── bwi
        ├── cmh
        ├── den
        ├── oma
        │   └── 2020
        │       ├── 01
        │       │   └── 01
        │       │       ├── 00
        │       │       ├── 01   
        │       │       ├── 02
        │       │       ...     
        │       │       ├── 22
        │       │       └── 23
        │       ├── 02
        │       ...
        │       └── 12
        └── sfo
```

In [28]:
sales_dirs = set()
sales_path = raw_data_dir.joinpath('sales')
sale_paths = [sales_path.joinpath(loc) for loc in locations]
def date_add_hours(date_list, hour_list):
    dates_plus_hours = set()
    for date in date_list:
        for hour in hour_list:
            comb_date_hour = date.joinpath(str(hour))
            dates_plus_hours.add(comb_date_hour)
    return dates_plus_hours

# TODO: Create and add the paths for this data set

for loc_path in sale_paths:
    temp_loc_date_set = gen_date_paths(curr_days, loc_path)
    temp_loc_dt_set = date_add_hours(temp_loc_date_set, range(24))
    sales_dirs |= temp_loc_dt_set

make_date_paths(sales_dirs)

# Should have 43,800 directories (43,920 if leap year)
len(sales_dirs) 

43800

### Assignment 5.3

#### Assignment 5.3.a

We have two choices for structuring our weekly data set. We can use the following naming convention where the date is based on the first day of the week. 

```
/data/processed/modeling/<year>/<month>/<day>/
```

Otherwise, we could use a naming convention where *week* is the number of weeks it has been since the beginning of the year. 
 
```
/data/processed/modeling/<year>/<week>/
```

We will use the first option for our naming convention. Python's *calendar* library has a function that determines the first day of the week. 

In [34]:
modeling_data_dirs = set()
modeling_path = processed_data_dir.joinpath('modeling')

# TODO: Create and add the paths for this data set

modeling_dirs= gen_date_paths(curr_days, modeling_path, day_of_week=True)
modeling_data_dirs |= modeling_dirs
make_date_paths(modeling_data_dirs)


# Should have 52 directories

#I just wanted to take a moment to point out there is a discrepency in the expected output vs the above request. If we do the first option we should have 12*7 directories (or 84)
#The second option (# of weeks since start of year) then we would have the expected 52 directories.
len(modeling_data_dirs) 

84

#### Assignment 5.3.b

Next, create the monthly inventory requests using the following convention. 

```
/data/processed/inventory/requests/<year>/<month>/
```

In [36]:
inventory_request_dirs = set()
inventory_request_path = processed_data_dir.joinpath('inventory/requests')
# TODO: Create and add the paths for this data set
def gen_month_paths(year_list, base_path):
    month_set = set()
    for year in year_list:
        year_month_list = set([base_path.joinpath(f'{year}/{i}') for i in range(1,13)])
        month_set |= year_month_list
    return month_set

inventory_request_dirs |= gen_month_paths([current_year], inventory_request_path)
make_date_paths(inventory_request_dirs)
 # Should output 12 directories
sorted(list(inventory_request_dirs))

[WindowsPath('data/processed/inventory/requests/2023/1'),
 WindowsPath('data/processed/inventory/requests/2023/10'),
 WindowsPath('data/processed/inventory/requests/2023/11'),
 WindowsPath('data/processed/inventory/requests/2023/12'),
 WindowsPath('data/processed/inventory/requests/2023/2'),
 WindowsPath('data/processed/inventory/requests/2023/3'),
 WindowsPath('data/processed/inventory/requests/2023/4'),
 WindowsPath('data/processed/inventory/requests/2023/5'),
 WindowsPath('data/processed/inventory/requests/2023/6'),
 WindowsPath('data/processed/inventory/requests/2023/7'),
 WindowsPath('data/processed/inventory/requests/2023/8'),
 WindowsPath('data/processed/inventory/requests/2023/9')]

#### Assignment 5.3.c

Finally, create the quarterly financial reports using the following convention. 

```
`/data/processed/financials/quarterly/<year>/<quarter>/`
```
While it does not matter for this assignment, the following are the typical dates associated with financial quarters. 

In [37]:
financials_dirs = set()
financial_path = processed_data_dir.joinpath('financials/quarterly')

# TODO: Create and add the paths for this data set
def gen_qtr_paths(year_list, base_path):
    qtr_set = set()
    for year in year_list:
        year_qtr_list = set([base_path.joinpath(f'{year}/{i}') for i in range(1,5)])
        qtr_set |= year_qtr_list
    return qtr_set

financials_dirs |= gen_qtr_paths([current_year], financial_path)
# Should output four quarterly directories
sorted(list(financials_dirs)) 

[WindowsPath('data/processed/financials/quarterly/2023/1'),
 WindowsPath('data/processed/financials/quarterly/2023/2'),
 WindowsPath('data/processed/financials/quarterly/2023/3'),
 WindowsPath('data/processed/financials/quarterly/2023/4')]